In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cargo Dataset principal

In [2]:
df = pd.read_csv('../../data/Top100_prediction_con.csv')

In [3]:
# Lista para guardar resultados
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for producto in productos:
    # Filtrar el DataFrame para el producto actual
    df_filtrado = df_original[(df_original['store_code'] == 'NYC_1') & (df_original['item'] == producto)]
    
    # Verificar si hay suficientes datos
    if len(df_filtrado) < 30 + 14:  # Necesitas al menos 44 registros
        print(f"Advertencia: No hay suficientes datos para el producto {producto}. Saltando...")
        continue
    
    # Preparar los datos
    ts_predict = df_filtrado[['date', 'income', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separar en entrenamiento y prueba
    y, exog = ts_predict['income'], ts_predict.drop(columns=['income'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Inicializar el modelo Forecaster
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor( random_state=105,
                                        max_depth=10,
                                        min_samples_leaf=2,
                                        min_samples_split=5,
                                        n_estimators=50),
        lags=7
    )
    
    # Ajustar el modelo
    forecaster.fit(y=y_train, exog=exog_train)
    
    start_date = '2016-04-25' 
    end_date ='2016-05-30'
    # Crear un rango de fechas
    date_range = pd.date_range(start=start_date, end=end_date)
    # Crear el dataframe
    df1 = pd.DataFrame(date_range, columns=['date'])
    df1['date'] = pd.to_datetime(df1['date'])
    df1['weekday'] = df1['date'].dt.day_name()
    df1['event'] = 0
    df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
    df1.sort_values('date', inplace=True, ascending=True)
    df1.set_index('date', inplace=True)
    df1 = df1.asfreq('D')
    exog_test = pd.concat([exog_test, df1], ignore_index=False)

    # Predecir
    predicciones = forecaster.predict(steps=60, exog=exog_test)
    
    # Almacenar resultados
    resultados.append({
        'producto': producto,
        'prediccion': predicciones.values,
        'test': y_test.values
    })
    
    #print(f"Predicciones generadas para el producto: {producto}")


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider 

In [4]:
tiendas_inventario = pd.DataFrame(resultados)
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
suma = tiendas_inventario['test_suma'].sum()
tiendas_inventario['prediccion'] = tiendas_inventario['prediccion'].apply(lambda x: np.round(x, decimals=0))
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[30:]))
tiendas_inventario[['producto','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

,producto,test_suma,tiendas_inventario_test,tiendas_inventario_prediction
0,ACCESORIES_1_108,483.3752,620.0,541.0
1,HOME_&_GARDEN_1_027,402.9750,590.0,615.0
2,HOME_&_GARDEN_1_053,2245.5000,1436.0,1368.0
3,HOME_&_GARDEN_1_140,823.3500,851.0,821.0
4,HOME_&_GARDEN_1_177,270.0875,816.0,799.0
...,...,...,...,...
95,SUPERMARKET_3_499,1262.5920,1241.0,1323.0
96,ACCESORIES_1_158,3789.8616,4156.0,4108.0
97,SUPERMARKET_3_282,2076.6720,2383.0,2756.0
98,ACCESORIES_1_354,7701.9768,6097.0,5711.0


In [5]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

118045.67990000002
127727.0
-9681.320099999983


In [4]:
tienda =  pd.DataFrame(resultados)
file_name = "NYC_1_ventas.xlsx"
tienda.to_excel(file_name)

In [6]:
pd.DataFrame(resultados)

,producto,prediccion,test
0,ACCESORIES_1_108,"[38.03304096666667, 29.221694174993765, 28.436...","[17.2634, 51.7902, 0.0, 0.0, 17.2634, 17.2634,..."
1,HOME_&_GARDEN_1_027,"[23.162962804012096, 21.291068008362185, 17.58...","[0.0, 14.924999999999995, 14.924999999999995, ..."
2,HOME_&_GARDEN_1_053,"[68.01256535517467, 65.18199440091247, 47.9109...","[112.275, 74.85, 130.98749999999998, 93.5625, ..."
3,HOME_&_GARDEN_1_140,"[33.854670403076504, 45.394227028713104, 27.87...","[37.425, 0.0, 56.1375, 18.7125, 18.7125, 18.71..."
4,HOME_&_GARDEN_1_177,"[30.34451217978839, 35.922654256910675, 22.001...","[26.137500000000003, 8.7125, 8.7125, 8.7125, 1..."
...,...,...,...
95,SUPERMARKET_3_499,"[34.55080285714286, 19.680890252100838, 37.579...","[0.0, 0.0, 55.296, 82.944, 46.08, 55.296, 101...."
96,ACCESORIES_1_158,"[178.17383305652177, 125.9383989409091, 130.25...","[122.2536, 336.1974, 122.2536, 244.5072, 152.8..."
97,SUPERMARKET_3_282,"[83.28954789479337, 102.44362450216448, 65.235...","[82.944, 52.224, 46.08, 67.584, 58.368, 52.224..."
98,ACCESORIES_1_354,"[283.29192071772786, 254.0419454932203, 180.93...","[397.3242, 244.5072, 244.5072, 183.3804, 244.5..."


## Predigo 30 dias(los de test) 